# XGBoost (Heart Data)

XGBoost는 앙상블(Ensemble) 기법을 구현한 대표적인 알고리즘은 Gradient Boost를 사용하며, Regression, Classification 모두에서 사용가능하다. 성능과 자원의 효율이 좋아 여러 문제에서 좋은 성과를 보여준다. 최근에는 LightGBM, Catboost 같은 보다 나은 모델이 등장하여 현업에서의 사용은 다소 줄어든 상황이다.

**XGBoost의 장점**
- GBM 대비 빠른 수행시간
- 병령 처리로 학습하여 속도가 빠르다.
- 과적합 규제(Regularization)를 포함하고 있어 강한 내구성을 가진다.
- Greedy Algorithm을 활용하여 가지치기(Pruning)하며, 이를 통해 과적합(Overfitting)에서 좀 더 자유롭다.
- CART(Classification And Regression Tree) 앙상블 모델을 사용
- Ealrly Stopping 기능 존재
- 유연성이 좋아 다양한 옵션과 Customizing이 용이하다.
- 결측치(Missing Values)를 내부적으로 처리하여 편리함을 제공

In [1]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt

%matplotlib inline

## 1. Load Datasets

In [2]:
heart_data = pd.read_csv('./data/Heart.csv', index_col=0)
heart_data.head(3)

,Age,Sex,ChestPain,RestBP,Chol,Fbs,RestECG,MaxHR,ExAng,Oldpeak,Slope,Ca,Thal,AHD
1,63,1,typical,145,233,1,2,150,0,2.3,3,0.0,fixed,No
2,67,1,asymptomatic,160,286,0,2,108,1,1.5,2,3.0,normal,Yes
3,67,1,asymptomatic,120,229,0,2,129,1,2.6,2,2.0,reversable,Yes


In [3]:
heart_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 303 entries, 1 to 303
Data columns (total 14 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Age        303 non-null    int64  
 1   Sex        303 non-null    int64  
 2   ChestPain  303 non-null    object 
 3   RestBP     303 non-null    int64  
 4   Chol       303 non-null    int64  
 5   Fbs        303 non-null    int64  
 6   RestECG    303 non-null    int64  
 7   MaxHR      303 non-null    int64  
 8   ExAng      303 non-null    int64  
 9   Oldpeak    303 non-null    float64
 10  Slope      303 non-null    int64  
 11  Ca         299 non-null    float64
 12  Thal       301 non-null    object 
 13  AHD        303 non-null    object 
dtypes: float64(2), int64(9), object(3)
memory usage: 35.5+ KB


## 2. PreProcessing

In [4]:
# Null 값 데이터 제거
heart_data = heart_data.dropna()
heart_data.shape

(297, 14)

In [5]:
# X, y갑 분할
X = heart_data[heart_data.columns.difference(['AHD'])]
y = heart_data['AHD']

# 학습, 테스트 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=123, test_size=0.2)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(237, 13) (237,)
(60, 13) (60,)


In [7]:
# dummies
X_train = pd.get_dummies(X_train)
X_test = pd.get_dummies(X_test)

X_train.head(3)

,Age,Ca,Chol,ExAng,Fbs,MaxHR,Oldpeak,RestBP,RestECG,Sex,Slope,ChestPain_asymptomatic,ChestPain_nonanginal,ChestPain_nontypical,ChestPain_typical,Thal_fixed,Thal_normal,Thal_reversable
54,44,0.0,219,0,0,188,0.0,130,2,1,1,0,0,1,0,0,1,0
268,59,1.0,218,0,1,134,2.2,126,0,1,2,0,1,0,0,1,0,0
97,59,1.0,239,1,0,142,1.2,110,2,1,2,1,0,0,0,0,0,1


## 3. XGBoost (Extream Gradient Boosting)

XGBoost와 Random Forest의 성능을 비교해 본다.

----

**TIP)**   
*XGBoost의 기본 원리*

XGBoost는 Boosting 기법을 사용하고 Boosting 기법은 약한 분류기를 조합하여 보다 높은 정확도를 이끌어내는 기법이다. 예를 들어 어떤 학습기 $M$에 대하여 $Y$를 예측할 확률은 $Y = M_{x_{1}} + error_{(1)}$로 표현 할 수 있는데, error를 좀 더 상세하게 분류 할 수 있는 $G$모델이 있다면 $error_{(1)}$은 $error_{(1)} = G_{x_{2}} + error_{(2)}$로 표현 할 수 있다. 마찬가지로 $error_{(2)}$는 임의의 모델 $H$에 의해 $error_{(2)} = H_{x_{3}} + error_{(3)}$로 표현된다.

결론적으로, $Y$는 아래의 식으로 표현 될 수 있다.  
$Y = M_{x_{1}} + G_{x_{2}} + H_{x_{3}} + error_{(3)}$ $(단, error_{(1)} > error_{(2)} > error_{(3)})$

이는, $M$ 하나의 모델만을 사용할 때 보다는 높은 결과를 얻을 수 있겠지만 같은 비중으로 모델 $M$, $G$, $H$를 처리하게 되면 서로 모델이 간섭하여 좋은 결과를 얻기 힘드므로, 각 모델의 가중치를 반영하고 그 결과는 다음과 같다.   
$Y = W_{1}M_{x_{1}} + W_{2}G_{x_{2}} + W_{3}H_{x_{3}} + error_{(3)}$   

----